In [ ]:
!unzip trainingsdata.zip

Archive:  trainingsdata.zip
  inflating: y_train.npy             
  inflating: y_val.npy               
  inflating: X_test.csv              
  inflating: X_train.csv             
  inflating: X_val.csv               
  inflating: y_test.npy              


In [ ]:
!unzip data.zip

Archive:  data.zip
  inflating: lda_train.npy           
  inflating: lda_val.npy             
  inflating: X_test_ft_avg.npy       
  inflating: X_test_ft_usif.npy      
  inflating: X_test_glove_avg.npy    
  inflating: X_test_glove_usif.npy   
  inflating: X_train_ft_avg.npy      
  inflating: X_train_ft_usif.npy     
  inflating: X_train_glove_avg.npy   
  inflating: X_train_glove_usif.npy  
  inflating: X_val_ft_avg.npy        
  inflating: X_val_ft_usif.npy       
  inflating: X_val_glove_avg.npy     
  inflating: X_val_glove_usif.npy    
  inflating: lda_test.npy            


# Imports
Here the imports are listed, which are important for the model

In [ ]:
!pip install gensim
!pip install -U git+https://github.com/oborchers/Fast_Sentence_Embeddings

  Cloning https://github.com/oborchers/Fast_Sentence_Embeddings to /tmp/pip-req-build-1ay5f8vw
  Running command git clone -q https://github.com/oborchers/Fast_Sentence_Embeddings /tmp/pip-req-build-1ay5f8vw
     |████████████████████████████████| 24.2 MB 93 kB/s 
     |████████████████████████████████| 56.8 MB 25 kB/s 
  Using cached Cython-0.29.14-cp37-cp37m-manylinux1_x86_64.whl (2.1 MB)
     |████████████████████████████████| 168 kB 72.0 MB/s 
     |████████████████████████████████| 721 kB 38.7 MB/s 
  Created wheel for fse: filename=fse-0.1.16-cp37-cp37m-linux_x86_64.whl size=656713 sha256=81c900be5d5adc9ca7d4c4c6e0da9b7391c4c42f58f9798597b7b99a060e1672
  Stored in directory: /tmp/pip-ephem-wheel-cache-l9vcaw8s/wheels/2a/d4/e6/644ae9126bf0fd4d3c5059e5b49d2728e080b2dc389cad953b
  Created wheel for wordfreq: filename=wordfreq-2.5.0-py3-none-any.whl size=56830860 sha256=6eacde4aed5c33daddc6c918ce421a19f314735bde5f3871f9a87f570641d08d
  Stored in directory: /root/.cache/pip/wheels/2f/

In [ ]:
import numpy
import sklearn
from sklearn import metrics
from sklearn.model_selection import KFold
import tensorflow as tf
#from tensorflow.keras.layers.experimental import preprocessing
from tensorflow import keras
import keras.backend as K
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.decomposition import NMF, LatentDirichletAllocation

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import tensorflow_hub as hub

#logger
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

#type annotation
from typing import *
tf.get_logger().setLevel('ERROR')

import gc
gc.enable()

SEED = 100    #reproduceability
from tensorflow.keras.layers import Embedding

## Vectorisation (if needed)
If already vectorised documents are used, this step can be ignored

### Bow vectoriser and Sentence encoder

In [ ]:
def getVectoriser(kind:str = "tfIdf", vocab:number=512, ngram_range=(1,1), trainingsData=None):
  """This function creates the vectoriser according to the parameter and returns it.
      It can be either a count, tfIdf or sentence vectoriser.

    Args:
        kind (str): The kind of the vectoriser such as 'tfIdf', 'sentence' or 'count'
        vocab (number): The length of the vocabulary
        ngram_range (any): The ngram range
        trainingsData (any): The trainingsdata

    Returns:
        The vectoriser object
    """
  global vectoriser_func
  if kind is "tfIdf":
    vectoriser = TfidfVectorizer(lowercase=True, min_df=2, max_features=vocab, ngram_range=ngram_range, stop_words="english")
    trainVectoriser(vectoriser, trainingsData)
    return vectoriser
  elif kind is "count":
    vectoriser = CountVectorizer(lowercase=True, min_df=2, max_features=vocab, ngram_range=ngram_range, stop_words="english")
    trainVectoriser(vectoriser, trainingsData)
    return vectoriser
  elif kind is "sentence":
    return hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")


def trainVectoriser(vectoriser, trainingsData):
  """This function is used to train the vectoriser

    Args:
        trainingsData (any): The data used for training
        vectoriser (any): The vectoriser itself

    Returns:
        The vectorised data.
    """
  
  text_ds = trainingsData.flatten()
  tmp = vectoriser.fit_transform(text_ds)
  logger.info(tmp.shape)
  del text_ds, tmp
  gc.collect()

def batchVectorisation(dfData, vectoriser, kindOfvectoriser:str):
  """This function loads the data in batches to reduce RAM requirenments.
    Also it dstinguishes between 'sentence' for the sentence encoder and plain sklearn
    vectorisers

    Args:
        dfData (any): The data of the dataframe
        vectoriser (any): The vectoriser itself
        kindOfvectoriser (str): This is the 'name' of the vectoriser, such as tfidf

    Returns:
        The vectorised data.
    """
  
  if kindOfvectoriser is "sentence":
    vecFunc = lambda string: vectoriser(tf.constant(string)) 
  else:
    vecFunc = lambda string: vectoriser.transform(string).toarray()
  l = dfData[:,0]
  r = dfData[:,1]
  returnValue = [[],[]]
  batch_size = 250
  while len(l) >0 :
    le = vecFunc(l[:batch_size]) #vectoriser.transform(l[:batch_size]).toarray()
    re = vecFunc(r[:batch_size]) #vectoriser.transform(r[:batch_size]).toarray()

    if len(returnValue[0]) == 0:
      returnValue[0] = le
      returnValue[1] = re
    else:
      returnValue[0] = numpy.concatenate((returnValue[0], le), axis=0)
      returnValue[1] = numpy.concatenate((returnValue[1], re), axis=0)

    l = l[batch_size:]
    r = r[batch_size:]
    logger.info(f"{returnValue[0].shape} todo {l.shape}")
  return returnValue

def vectoriseData(vectoriser, training, validation, kind:str = "tfidf"):
  """This function vectorises the data provided given the vectoriser provided.

    Args:
        vectoriser (any): This is the vectoriser object, it can be either be the
        universal setnence encoder or one of the bow vectorisers
        training (any): Is the trainingsdata data
        validation (any): Is the validation data
        kind (str): Is the data proviced

    Returns:
        The list of tokenised documents
    """
  X_train_list = batchVectorisation(training, vectoriser, kind)
  X_val_list = batchVectorisation(validation, vectoriser, kind)
  return X_train_list, X_val_list

### GloVe; Fasttext usif and avg document embeddings

In [ ]:
from gensim.models import Word2Vec, FastText
!pip install -U fse
from fse.models.average import FAST_VERSION
print(FAST_VERSION) #1 means available
from fse.models import Average, uSIF
import gensim.downloader as api
from fse import IndexedList

class EmbeddingModels():
  """This class is used to create a Embedding model, which is able to vectorise documents.
    Returns:
        The document embeddings producing object
    """
  def __init__(self, vocab_len, trainingsdata, embedName, combiName):
    self.tokenizer = TfidfVectorizer(lowercase=True, min_df=2, max_features=vocab_len, ngram_range=(1,1), stop_words="english").build_tokenizer()
    tokdata = self.tokenize(trainingsdata)

    self.embed = self.get_embedding(tokdata, embedName, vocab_len)
    logger.error("got embedding")

    tokdata = IndexedList(tokdata)
    logger.error("csp")

    self.model = self.getMergingMethod(combiName, self.embed, embedName, vocab_len)
    logger.error("got combi")
    self.model.train(tokdata)
    logger.error("finished training")
  
  def getMergingMethod(self, combiName:str, embedding, embedName:str, voc_len:number):
    """This function returns the merging method for the word embeddings.

    Args:
        embedding (any): Serves the word embeddings
        embedName (str): is the name of the embeddings
        voc_len (number): is the lenght of the vocabulary
        combiName (str): embedName is the name of the embeddings

    Returns:
        The Merging method
    """
    if combiName is "avg":
      return Average(embedding)
    else:
      if embedName is "glove":
        return uSIF(embedding, workers=2, lang_freq="en")
      else:
        return uSIF(embedding, length=voc_len)

  def get_embedding(self, trainingsdata, embedName:str, vocab_len:number):
    """This function creates and returns an embedding for the given trainingsdata.

    Args:
        trainingsdata (List[str]): Is the data proviced
        embedding (any): Serves the word embeddings
        embedName (str): is the name of the embeddings
        vocab_len (number): is the lenght of the vocabulary
        combiName (str): embedName is the name of the embeddings

    Returns:
        The word level embedding function 
    """
    if embedName is "glove":
      return api.load("glove-wiki-gigaword-100")
    elif embedName is "fasttext":
      return FastText(window=3, min_count=1, sentences=trainingsdata, size=vocab_len)
    else:
      return Word2Vec(trainingsdata, min_count=1, size=vocab_len)

  def tokenize(self, data)->list:
    """This function tokenises all the documents in the data.

    Args:
        data (any): Is the data proviced

    Returns:
        The list of tokenised documents
    """
    return [self.tokenizer(doc) for doc in data]

  def predict(self, data)->list:
    """This function creates the document embeddings

    Args:
        data (any): Is the data proviced

    Returns:
        The document embeddings
    """
    tokenized_data = self.tokenize(data)
    idxTok = IndexedList(tokenized_data)
    inf_tokens = list(map(lambda tokenList: (tokenList, 0), tokenized_data))
    #print(numpy.array(inf_tokens))
    return list(map(lambda inf_data: self.model.infer([inf_data])[0], inf_tokens))

## LDA

In [ ]:
class NumberNormalizingVectorizer(TfidfVectorizer):
  """This class creates a custom vectoriser used for the LDA.

    Returns:
        A object of the Vectoriser
    """
  def build_tokenizer(self):
    tokenize = super().build_tokenizer()
    return lambda doc: list(self.number_normalizer(tokenize(doc)))

  def number_normalizer(self, tokens):
    return ("NUMBER" if token[0].isdigit() else token for token in tokens)

class GenerateLdaTopics():
  """
  This Class is used to generate the LDA
  """
  def __init__(self, _n_topics:number=200):
    self.n_topics = _n_topics
    self.cv = NumberNormalizingVectorizer(lowercase=True, min_df=5, max_features=20000, max_df=0.5, stop_words="english",token_pattern='[a-zA-Z]+')
    self.lda = LatentDirichletAllocation(n_components=_n_topics, random_state=0,n_jobs=-1)
    self.vocab = None
    
  def fit(self, data):
    """This function is used to fit the LDA model.

    Args:
        data (any): The data passed to the vectoriser
    """
    vectorised_data = self.cv.fit_transform(X_train.flatten())
    logger.info("data vectorised")
    self.vocab = self.cv.get_feature_names()
    logger.info("fitting lda")
    self.lda.fit(vectorised_data)

  def predict(self, data):
    """This function returns the topic propabilities for the documents.

    Args:
        data (any): The data to produce the document-topic matri from

    Returns:
        The document-topic matrix
    """
    vec_data = self.cv.transform(data)     #document-term matrix
    return self.lda.transform(vec_data)    #document-topic matrix

  def print_topics(self,n_top_words:number=5):
    """In order to visualise the topics,
    this document prints representatives of the topics to show which words are contained within each topic.

    Args:
        n_top_words (number): The number words displayed for the topics

    Returns:
        A trainable model
    """
    for topic, comp in enumerate(self.lda.components_):  #iterate through the topic-term matrix
      word_idx = numpy.argsort(comp)[:-n_top_words-1:-1]
      words = [self.vocab[i] for i in word_idx]
      print('Topic: %d' % topic)
      print('  %s' % ', '.join(words)) 

  def print_predictedTopics(self, data, n_top_topics:number = 5):
    """This function appends the vector length onto the vector itself.

    Args:
        data (number): The length of the documents
        n_top_topics (number): The number of topics

    Returns:
        A trainable model
    """
    pred = self.predict(data)
    topicsAndPropabilities = []
    for p in pred:    #for document in document-topic
      top_topics = p.argsort()[:-n_top_topics - 1:-1]
      propabilities = p[top_topics]
      topicsAndPropabilities.append([top_topics, propabilities])#print k best topics
    return topicsAndPropabilities

#The classifier
To change the classifier, the concatenation layer in the *createModel* function has to be edited.


In [ ]:
class LayerAppendSum(tf.keras.layers.Layer):
  """This function appends the vector length onto the vector itself.
    """
  def __init__(self):
      super(LayerAppendSum, self).__init__()

  def call(self, inputs):
    """This function gets automatically called with the data.

    Args:
        inputs (any): A vector within a vector of all the batches

    Returns:
        A concatenated vector length onto the vector 
    """
    def mapfunc(vec):
      reduced_sum = tf.reduce_sum(vec)
      reshaped = tf.reshape(reduced_sum, [-1])
      return tf.concat([vec, reshaped], 0)
    return tf.map_fn(lambda batch: mapfunc(batch), inputs)  #tf.concat([inputs, reduced])


def createModel(vec_len:int):
  """This function creates a classifier for feature lists of the vec_len provided

    Args:
        vec_len (int): The length of the documents

    Returns:
        A trainable model
    """
  ain = keras.Input(shape=(vec_len,), dtype="float")
  bin = keras.Input(shape=(vec_len,), dtype="float")

  a = tf.keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=0))(ain)
  b = tf.keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=0))(bin)

  avg_layer = tf.keras.layers.Average(name="avg")([a, b])
  mult_layer = tf.keras.layers.Multiply(name="mult")([a,b])
  sub_layer = tf.keras.layers.Subtract(name="sub")([a, b])

  cosine_sim = tf.keras.layers.Dot(axes=1, normalize=True, name="cosine_similarity")([a,b])
  #topic_sim = tf.keras.layers.Dot(axes=1, normalize=True, name="topic_similarity")([x,y])

  #avg_layer = LayerAppendSum()(avg_layer)
  #mult_layer = LayerAppendSum()(mult_layer)
  #sub_layer = LayerAppendSum()(sub_layer)

  hidden = tf.keras.layers.concatenate([sub_layer, avg_layer, mult_layer,cosine_sim])#a,b , topic_sim])

  outputs = layers.Dense(5, activation='softmax')(hidden)

  model = keras.Model([ain,bin], outputs)   #,x,y
  model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])
  #model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy']),
  return model

def download_model(model):
  """
  This function is used to download models from google collab specifficly.
  !tar -czvf model.tar.gz model/ is used for tarring the model
  !tar -xvzf model.tar.gz is used for ent-tarring the model
  """
  model.save(f"./tmp_model")
  !tar -czvf tmp_model.tar.gz tmp_model/
  from google.colab import files
  files.download('tmp_model.tar.gz')

def eval_model(predLabel:List[int], true_label:List[int]):
  """This function is used to determine the precision, recall and f1 score of a given model.

    Args:
        predLabel (List[int]): The predicted label of the model
        predLabel (List[int]): The gold standard label

    Returns:
        The precission, recall, f1, and f1_macro scores
    """
  f1_scores = sklearn.metrics.f1_score(y_true=true_label, y_pred=predLabel, average=None)
  precission = sklearn.metrics.precision_score(y_true=true_label, y_pred=predLabel, average=None)
  recall = sklearn.metrics.recall_score(y_true=true_label, y_pred=predLabel, average=None)

  precission = list(map(lambda x: round(x, 4), precission))
  recall = list(map(lambda x: round(x, 4), recall))
  f1 = list(map(lambda x: round(x, 4), f1_scores))
  f1_macro =  round(sum(f1)/len(f1), 4)
  return precission, recall, f1, f1_macro

## Running the model configuration

In [ ]:
#newLda = GenerateLdaTopics()
#newLda.fit(X_train.flatten())

INFO:root:data vectorised
INFO:root:fitting lda


#### Finding the best vectoriser configuration

In [ ]:
def loadTrainAndValFeatures(nameTrainDataset: str, nameValDataset: str) -> list:
  """This function loads the training and validation data.

    Args:
        nameTrainDataset (str): The name of the training dataset
        nameValDataset (str): The name of the validation dataset

    Returns:
        Both, the training and validation dataset
    """
  X_train:numpy.ndarray = loadCSVData("X_train")
  X_val:numpy.ndarray = loadCSVData("X_val")

  return X_train, X_val

def loadCSVData(name:str) -> numpy.ndarray:
  """This function loads SCV give the name.

    Args:
        name (str): The name of the dataset

    Returns:
        The numpy array
    """
  return pd.read_csv(f"{name}.csv")[["IssueA", "IssueB"]].values.astype("str")

def loadNpData(name:str)-> numpy.ndarray:
  """This function loads numpy data give the name.

    Args:
        name (str): The name of the dataset

    Returns:
        The numpy array
    """
  return numpy.load(name+".npy", allow_pickle=True)

def loadTrainAndValResults(nameTrainDataset: str, nameValDataset: str):
  """This function loads the training and validation results.

    Args:
        nameTrainDataset (str): The name of the training dataset
        nameValDataset (str): The name of the validation dataset

    Returns:
        Both, the training and validation results
    """
  Y_train = loadNpData("y_train")
  Y_val = loadNpData("y_val")
  Y_val = numpy.array([numpy.array(x) for x in Y_val])
  Y_train = numpy.array([numpy.array(x) for x in Y_train])
  return Y_train, Y_val


def getVectorisedDataFromDataset(lda:bool, name:str, lda_name:str = None):
  """This function loads a specific X_train and X_val dataset and appends lda if needed.

    Args:
        lda (bool): Whether or not the model uses lds.
        name (str): The The name of the data to load.
        lda_name (str): The name of the lda datato load.

    Returns:
        The loaded data
    """
  tmp_list = list(loadNpData(name))

  if lda:
    lda_data = list(loadNpData(lda_name))
    tmp_list.append(lda_data[0])
    tmp_list.append(lda_data[1])

  return tmp_list

def createVectorisedData(name:str,ngram,voc:int, lda:bool, X_train, X_val):
  """This function creates vectorised data, by creating a vectoriser given the requirenments specified and transforms the data given.

    Args:
        lda (bool): Whether or not the model uses lds.
        name (str): The name of the data to load.
        voc (int): The length of the vocabulary.
        ngram (tuple): The ngram itself.
        X_train (any): The training data.
        X_val (any): The validation data.

    Returns:
        The training and validation data
    """
  newLda= None
  vectoriser = getVectoriser(trainingsData=X_train, vocab=voc, kind=name, ngram_range=ngram)
  X_train_list, X_val_list = vectoriseData(vectoriser, X_train, X_val, kind=name)
  if lda:
    newLda = GenerateLdaTopics()
    newLda.fit(X_train.flatten())
    tl = newLda.predict(X_train[:,0])
    tr = newLda.predict(X_train[:,1])
    vl = newLda.predict(X_val[:,0])
    vr = newLda.predict(X_val[:,1])

    X_train_list.append(tl)
    X_train_list.append(tr)
    X_val_list.append(vl)
    X_val_list.append(vr)

  return X_train_list, X_val_list
  
def createAndEvaluateModel(X_train_list:list, X_val_list:list, Y_train, Y_val, voc:int):
   """This function creates and evaluates a model.

    Args:
        X_train_list (list): The training data.
        X_val_list (list): The validation data.
        Y_train (numpy.ndarray): The training solution.
        Y_val (numpy.ndarray): The validation solution.
        voc (int): The vocabulary size.

    Returns:
        The f1 scores evaluated
    """
  model = createModel(voc)
  model.fit(x:list=X_train_list, y=Y_train, batch_size:int = 25, epochs:int=25, validation_data=(X_val_list, Y_val))

  output = model.predict(X_val_list, batch_size=15)
  output = list(map(numpy.argmax, output))
  y_val_int = list(map(numpy.argmax, Y_val))

  return eval_model(output, y_val_int)


def trainAndEvaluateModel(vectorisedData:bool, config, lda:bool):
  """
  This function creates and evaluates multiple models.

    Args:
        vectorisedData (bool): The training data.
        config (str): The validation data.
        lda (bool): Whether or not LDA was used.

    Returns:
        The best vectoriser vonfiguration for the model
    """
  precissions = []
  recalls = []
  f1s = []
  f1_macros = []
  Y_train, Y_val = loadTrainAndValResults("nameTrainDataset", "nameValDataset")
  
  if vectorisedData:
    for name in comfig:
      X_train_list = getVectorisedDataFromDataset(lda, f"X_train_{name}")
      X_val_list = getVectorisedDataFromDataset(lda, f"X_val_{name}")
      voc = len(X_train_list[0][0])

      precission, recall, f1, f1_macro = createAndEvaluateModel(X_train_list, X_val_list, Y_train, Y_val, voc)
      precissions.append(precission)
      recalls.append(recall)
      f1s.append(f1)
      f1_macros.append(f1_macro)
      gc.collect()

  else:
    for name,ngram,voc in config:
      X_train, X_val = loadTrainAndValFeatures("nameTrainDataset", "nameValDataset")
      X_train_list, X_val_list = createVectorisedData(name,ngram,voc, lda, X_train, X_val)

      precission, recall, f1, f1_macro = createAndEvaluateModel(X_train_list, X_val_list, Y_train, Y_val, voc)
      precissions.append(precission)
      recalls.append(recall)
      f1s.append(f1)
      f1_macros.append(f1_macro)
      gc.collect()
  
  npf1_macros = list(map(lambda x: round(x, 4), f1_macros))

  bestIndex = npf1_macros.index(max(npf1_macros))
  bestVecConfig = vectoriser_config[bestIndex]
  #print(f"\nBest model config:\n{bestVecConfig[0]}:{bestVecConfig[1]}:{bestVecConfig[2]} = {max(npf1_macros)}")
  print(f"\nBest model config:\n{bestVecConfig} = {max(npf1_macros)}")
  return bestVecConfig

#### Running the best vectoriser configuration
The best vectoriser vonfiguration for the model is then run on the test dataset for evaluation purposes

In [ ]:
def printLatexOutput(output:numpy.ndarray, Y_test_int:numpy.ndarray, name:str, ngram, voc:int):
  """
  This function creates a latex formated output to show the f1, precision, recall performances.

    Args:
        output (numpy.ndarray): The output of the classifier.
        Y_test_int (numpy.ndarray): The integer values of the gold labels
        name (str): The name of the vectorisation strategie.
        ngram (tuple): The ngram.
        voc (int): The vocabulary size.

    Returns:
        A latex formated output
    """
  output = list(map(numpy.argmax, output))
  precission, recall, f1, f1_macro = eval_model(output, Y_test_int)
  labels = numpy.array(['unrelated', 'duplicate', '<=>', '<=', '=>'])
  rearrange = numpy.array([4,3,2,0,1])
  labels = labels[rearrange]
  precission = numpy.array(precission)[rearrange]
  recall = numpy.array(recall)[rearrange]
  f1 = numpy.array(f1)[rearrange]

  ng = "" if ngram[1] == 1 else "_{,Un+Bigram}"
  output = f"{name}_{{{voc}}}{ng}" + "\t\t"
  for j in range(len(labels)):
    output += f"& {precission[j]} & {recall[j]} "
  print(output + "\\\\")
  output = f"{name}_{{{voc}}}{ng}" + "\t\t"

  print("\nf1 scores\t\t"+ "\t".join(labels) + "\tmacro f1")
  for j in range(len(labels)):
    output += f"& {f1[j]} "
  output += f" & {f1_macro}"
  print(output + "\\\\")

def vectoriseDataReturnVectoriser(name:str, ngram, voc:int, lda:bool):
  """
  This function loads the training data and returns it in additino to the vectoriser and a LDA if present.

    Args:
        name (str): The name of the vectorisation strategie.
        ngram (tuple): The ngram.
        voc (int): The vocabulary size.
        lda (bool): Whether or not a LDA model was used.

    Returns:
        The trainings and validation data in addition to an vectoriser, and LDA
    """

  X_train, X_val = loadTrainAndValFeatures("nameTrainDataset", "nameValDataset")
  Y_train, Y_val = loadTrainAndValResults("nameTrainDataset", "nameValDataset")

  vectoriser = getVectoriser(trainingsData=X_train, vocab=voc, kind=name, ngram_range=ngram)
  X_train_list, X_val_list = vectoriseData(vectoriser, X_train, X_val, kind=name)
  newLda = None
  if lda:
    newLda = GenerateLdaTopics()
    newLda.fit(X_train.flatten())
    tl = newLda.predict(X_train[:,0])
    tr = newLda.predict(X_train[:,1])
    vl = newLda.predict(X_val[:,0])
    vr = newLda.predict(X_val[:,1])

    X_train_list.append(tl)
    X_train_list.append(tr)
    X_val_list.append(vl)
    X_val_list.append(vr)
  return X_train_list, X_val_list, vectoriser, newLda

def loadAndVectoriseTestData(name:str, vectoriser, lda:bool, newLda):
  """
  This function cloads the test data and vectorises it using a given vectoriser.

    Args:
        name (str): The name of the vectorisation strategie.
        vectoriser (any): The vectoriser.
        newLda (any): The LDA model or None.
        lda (bool): Whether or not a LDA model was used.

    Returns:
        The vectorised data
    """
  x_test:numpy.ndarray = loadCSVData("X_test") 
  y_test:numpy.ndarray = loadNpData("y_test")

  X_test: List[numpy.ndarray] = batchVectorisation(x_test, vectoriser, name)#[test_dev[:,0],test_dev[:,1]]### list of numpy.ndarray[str]#
  if lda:
    tsl = newLda.predict(x_test[:,0])
    tsr = newLda.predict(x_test[:,1])
    X_test.append(tsl)
    X_test.append(tsr)

  Y_test_int:numpy.ndarray = numpy.array([numpy.argmax(numpy.array(tmp)) for tmp in y_test])
  return X_test, Y_test_int

def createBestModel(name:str,ngram,voc:int, lda:bool):
  """
  This function creates the 'best' model, using the data provided.

    Args:
        name (str): The name of the vectorisation strategie.
        ngram (tuple): The ngram.
        voc (int): The vocabulary size.
        lda (bool): Whether or not a LDA model was used.

    Returns:
        The 'best' model itself
    """
  X_train_list, X_val_list, vectoriser, newLda = vectoriseDataReturnVectoriser(name, ngram, voc, lda)

  model = createModel(voc)
  model.fit(x=X_train_list, y=Y_train, batch_size = 25, epochs=25, validation_data=(X_val_list, Y_val))

  del X_train, X_val, Y_train, Y_val
  gc.collect()

  X_test, Y_test_int = loadAndVectoriseTestData(name, vectoriser, lda, newLda)

  output = model.predict(X_test, batch_size=15)
  #print(numpy.array([f"{rnd(x)} {numpy.argmax(x)}" for x in output]))
  printLatexOutput(output, Y_test_int, name, ngram, voc)
  return model
 

def rnd(num:float)->float:
  """
  This function rounds the number given.

    Args:
        num (float): The number itself.

    Returns:
        The rounded number
    """
  return list(map(lambda x: round(x, 4), x))

vectoriser_config = [("count", (1,1), 512), ("tfIdf", (1,1), 512), ("count", (1,2), 512), ("tfIdf", (1,2), 512), 
                     ("count", (1,1), 1024), ("tfIdf", (1,1), 1024), ("count", (1,2), 1024), ("tfIdf", (1,2), 1024),
                     ("count", (1,1), 8192), ("tfIdf", (1,1), 8192), ("count", (1,2), 8192), ("tfIdf", (1,2), 8192),
                     ("count", (1,1), 16384), ("tfIdf", (1,1), 16384), ("count", (1,2), 16384), ("tfIdf", (1,2), 16384),
                     ("sentence",(1,1), 512)
                     ]

data_config = ["ft_avg", "glove_avg", "ft_usif", "glove_usif"]

name,ngram,voc = trainAndEvaluateModel(False, vectoriser_config, False)
model = createBestModel(name,ngram,voc, False)
#trainAndEvaluateModel(True, data_config, False)

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file="Embed_baseline.png",
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)